## Tiff export.
### 1) ImageJ Tiff export with ROI within ijmetadata.
### 2) OME Tiff export with ROI as OME-XML meta data.

ROI stored in OME XML metadata does not visualise well in Fiji/ImageJ, and the ImageJ ROI format is not properly recognised my OMERO. Therefore here we show both methods and give the user the option of which method they use for storing their data.


### ImageJ Tiff export with ROI as within ijmetadata

In [7]:
#For this we need the libary: https://github.com/dwaithe/ijpython_roi
from ijroi.ij_roi import Roi
from ijroi.ijpython_encoder import encode_ij_roi, RGB_encoder
import numpy as np
import tifffile
im_stk = np.zeros((10,1,3,300,512)).astype(np.uint16)
#TZCYX


data = []
roi_b = Roi(30, 40, 140, 120, im_stk.shape[3], im_stk.shape[4], 0)
roi_b.name = "Region 1"
roi_b.roiType = 1
channel =1
sliceZ = 6
frame = 1
roi_b.setPositionH(channel, sliceZ, frame)
roi_b.strokeLineWidth = 3.0
roi_b.strokeColor = RGB_encoder(255, 0, 255, 255)

data.append(encode_ij_roi(roi_b))

roi_b = Roi(130, 140, 140, 120, im_stk.shape[2], im_stk.shape[3], 0)
roi_b.name = "Region 1"
roi_b.roiType = 1
channel =2
sliceZ = 5
frame = 1
roi_b.setPositionH(channel, sliceZ, frame)
roi_b.strokeLineWidth = 3.0
roi_b.strokeColor = RGB_encoder(255, 0, 0, 255)

data.append(encode_ij_roi(roi_b))

metadata = {'hyperstack': True } 
metadata['mode'] = 'grayscale'
metadata['unit'] = 'um'
metadata['spacing'] = 0.5 #This is the z-spacing of the image-stack  (for unit see 'unit').
metadata['min']= 0.0
metadata['max']= 0.0
metadata['Info'] = 'Software: tifffile.py\n With ROI encoded using functions from https://github.com/dwaithe/ijpython_roi '
resolution = (1/0.26,1/0.26) #Expects tuple, ratio pixel to physical unit (for unit see 'unit').


tifffile.imsave("out5.tiff", im_stk,resolution=(1/0.26, 1/0.26),shape=im_stk.shape,ijmetadata={'Overlays':data},metadata=metadata,imagej=True)


self.slice 6
self.slice 5


In [8]:
#Allows exploration of meta data. Notice the tiff file meta data and ijmetadata are now mixed.
with tifffile.TiffFile("out5.tiff") as tif:
    imagej_hyperstack = tif.asarray()
    imagej_metadata = tif.imagej_metadata
print(imagej_metadata)

{'ImageJ': '1.11a', 'images': 30, 'channels': 3, 'frames': 10, 'hyperstack': True, 'mode': 'grayscale', 'loop': False, 'unit': 'um', 'spacing': 0.5, 'min': 0.0, 'max': 0.0, 'info': 'Software: tifffile.py', 'Overlays': [b'Iout\x00\xe3\x01\x00\x00(\x00\x1e\x00\xa0\x00\xaa\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x06\x00\x00\x00\x01\x00\x00\x00\x80\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00R\x00e\x00g\x00i\x00o\x00n\x00 \x001', b'Iout\x00\xe3\x01\x00\x00\x8c\x00\x82\x01\x04\x00\x85\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

### 2) OME Tiff export with ROI as OME-XML metadata.

In [13]:
from aicsimageio.vendor import omexml #https://github.com/AllenCellModeling/aicsimageio
import xml.etree.ElementTree as ElementTree
import sys
sys.path.append("../src/")
import ome_xml_roi_store as oxrs
##This is where I am defining the ROI for this image.
_roi = [] 
#example: _roi.append([ID,X,Y,Height,Width,StrokeColor,Text,TheZ,TheT,TheC])
_roi.append(["Shape:0:0",50,40,100,200,-65281,"0005-0126-0224",8,1,2])
_roi.append(["Shape:1:0",75,40,100,200,-65281,"0010-0126-0224",4,1,2])


#This is where I am defined text ROI for this image.
_roi_txt = []    
#_roi_txt.append([ID,X,Y,FontSize,FontSizeUnit,StrokeColor,Text, TheZ,TheT,TheC])
_roi_txt.append(["Shape:4:0",50,40,24,"pixel","-1","c=003, z=003, t=001, i=0009",3,1,2])
#_roi_txt.append(["Shape:5:0",50,40,24,"pixel","-1","c=003, z=003, t=001, i=0009",13,0,2])
#_roi_txt.append(["Shape:6:0",50,40,24,"pixel","-1","c=003, z=003, t=001, i=0009",13,0,2])


#An image.
im_stk = np.zeros((3, 10,1,300,512)).astype(np.uint16)
#C,Z,T,Y,X
num_x = im_stk.shape[4]
num_y = im_stk.shape[3]
num_z = im_stk.shape[1]
num_c = im_stk.shape[0]
num_t = im_stk.shape[2]

o = omexml.OMEXML() # Many different options for this. Check the 
o.image().Pixels.channel_count = 3
o.image().Pixels.Channel(0).Name = "Green"
o.image().Pixels.Channel(1).Name = "Blue"
o.image().Pixels.Channel(2).Name = "Red"
o.image().Pixels.set_DimensionOrder("XYCZT")
o.image().Pixels.set_SizeX(num_x)
o.image().Pixels.set_SizeY(num_y)
o.image().Pixels.set_SizeZ(num_z)
o.image().Pixels.set_SizeT(num_t)
o.image().Pixels.set_SizeC(num_c)
o.image().Pixels.set_PixelType("uint16")
o.image().Pixels.PhysicalSizeX = 0.26
o.image().Pixels.PhysicalSizeY = 0.26
o.image().Pixels.PhysicalSizeXUnit = "µm"
o.image().Pixels.PhysicalSizeYUnit = "µm"
o.image().Pixels.PhysicalSizeZUnit = "µm"



o.ct = 0

oxrs.create_roi(_roi,o)
oxrs.create_roi_txt(_roi_txt,o)
oxrs.create_ROI_ref(o)

o.image().Pixels.populate_TiffData()

    

meta_to = o.to_xml().encode()
tifffile.imsave("out75.ome.tiff", im_stk,shape=im_stk.shape, description=meta_to,bigtiff=True)


In [4]:
from aicsimageio import AICSImage

# Get an AICSImage object, which allows to read the OME metadata.
img = AICSImage("out75.ome.tiff")
print(img.metadata )

<?xml version='1.0' encoding='utf-8'?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/ome/2013-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" UUID="urn:uuid:c19074dc-71bf-4301-b4e8-b2b4c548d054" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2013-06 http://www.openmicroscopy.org/Schemas/OME/2012-03/ome.xsd">
  <Image ID="Image:0" Name="default.png">
    <AcquisitionDate>2020-02-17T11:01:14.594477</AcquisitionDate>
    <Pixels BigEndian="true" DimensionOrder="XYCZT" ID="Pixels:0" PhysicalSizeX="0.26" PhysicalSizeY="0.26" SizeC="3" SizeT="1" SizeX="512" SizeY="300" SizeZ="10" Type="uint16">
<Channel ID="Channel:0:0" Name="Green" SamplesPerPixel="1">
        <LightPath />
      </Channel>
    <Channel ID="700592ac-ad9a-41e2-9f68-d5c99fafbce4" Name="Blue" SamplesPerPixel="1" /><Channel ID="d2a79a9e-310c-4021-8a87-e9c009242c1c" Name="Red" SamplesPerPixel="1" /><TiffData FirstC="0" FirstT="0" FirstZ="0" IFD="0" PlaneCount="1" /><TiffData FirstC="1" FirstT="0" Fir

In [ ]:
#For this we need the libary: https://github.com/dwaithe/ijpython_roi
from ijroi.ij_roi import Roi
from ijroi.ijpython_encoder import encode_ij_roi, RGB_encoder
import numpy as np
import tifffile
im_stk = np.zeros((10,1,3,300,512)).astype(np.uint16)
#TZCYX


data = []
roi_b = Roi(30, 40, 140, 120, im_stk.shape[3], im_stk.shape[4], 0)
roi_b.name = "Region 1"
roi_b.roiType = 1
channel =1
sliceZ = 6
frame = 1
roi_b.setPositionH(channel, sliceZ, frame)
roi_b.strokeLineWidth = 3.0
roi_b.strokeColor = RGB_encoder(255, 0, 255, 255)

data.append(encode_ij_roi(roi_b))

roi_b = Roi(130, 140, 140, 120, im_stk.shape[2], im_stk.shape[3], 0)
roi_b.name = "Region 1"
roi_b.roiType = 1
channel =2
sliceZ = 5
frame = 1
roi_b.setPositionH(channel, sliceZ, frame)
roi_b.strokeLineWidth = 3.0
roi_b.strokeColor = RGB_encoder(255, 0, 0, 255)

data.append(encode_ij_roi(roi_b))

metadata = {'hyperstack': True } 
metadata['mode'] = 'grayscale'
metadata['unit'] = 'um'
metadata['spacing'] = 0.5 #This is the z-spacing of the image-stack  (for unit see 'unit').
metadata['min']= 0.0
metadata['max']= 0.0
metadata['Info'] = 'Software: tifffile.py\n With ROI encoded using functions from https://github.com/dwaithe/ijpython_roi '
resolution = (1/0.26,1/0.26) #Expects tuple, ratio pixel to physical unit (for unit see 'unit').


tifffile.imsave("out5.tiff", im_stk,resolution=(1/0.26, 1/0.26),shape=im_stk.shape,ijmetadata={'Overlays':data},metadata=metadata,imagej=True)


In [11]:
data = np.random.randint(0, 255, (5, 2, 3, 301, 219), 'uint8')
with tifffile.TiffWriter('temp.tif', bigtiff=True) as tif:
    for i in range(data.shape[0]):
        tif.save(data[i], compress=6, photometric='minisblack')